In [1]:
import os
import numpy as np
import pandas as pd
import glob
import json
from scipy import stats
import matplotlib.pyplot as plt
import sklearn.metrics as metric
from tqdm.notebook import tqdm
import src.online_decomposition.slidingSTL as slidingSTL     # STL in online mode (SlidingSTL)
import src.online_decomposition.ASTD as ASTD
import src.utilities.utility_stl as utility_stl
import statsmodels.api as sm

## Prepare data

In [2]:
FOLDER_DATASET = os.path.expanduser("~/source_code/SSTD_PKDD/dataset/02_Real1_datasets")
json_files = glob.glob(os.path.join(FOLDER_DATASET,'*.json'))

bounds_other = np.array([0.70, 0.80, 0.9, 1, 1.10, 1.20, 1.30])
bound_lynx = np.array([7, 8, 9, 10, 11, 12, 13])

results = [] # all results

In [8]:
for json_file in json_files:
    with open(json_file) as f:
        json_data = json.load(f)
    
    ts_data = np.array(json_data['ts'])
    ts_data = utility_stl.Znormalization(ts_data)
    ans_season_length = max(json_data['main_length'])
    
    if 'lynx' in os.path.basename(json_file):
        season_lengths = bound_lynx
        bounds = np.array([0.70, 0.80, 0.9, 1, 1.10, 1.20, 1.30])
    else:
        season_lengths = np.round(bounds_other * ans_season_length)
        bounds = bounds_other
    file_name = os.path.basename(json_file).replace('.json','')
    pbar = tqdm(total=len(season_lengths), desc=f"file_name")
    for season_length,bound in zip(season_lengths,bounds):
        season_length = int(season_length)
        ts_offline_phase = np.array(ts_data[:season_length * 5])
        ts_online_phase = np.array(ts_data[season_length * 5:])
        
        core_slidingSTL = slidingSTL.slidingSTL(max_season_length=season_length, max_cycles =5)
        slidingSTL_trend, slidingSTL_seasonal, slidingSTL_residual = core_slidingSTL.initialize_phase(ts_offline_phase)
        
        for y_t in ts_online_phase:
            t_t, s_t, r_t = core_slidingSTL.online_update(y_t)
            slidingSTL_trend = np.append(slidingSTL_trend, t_t)
            slidingSTL_seasonal = np.append(slidingSTL_seasonal, s_t)
            slidingSTL_residual = np.append(slidingSTL_residual, r_t)
        
        
        """
        OneShotSTL cannot export trend, seasonal, and residual components of the initialization phase. We must compute data that are processed in the online phase only for fair comparisons.
        The initialization phase of OneShotSTL require data with 5 * season length.
        """
        
        trend = slidingSTL_trend[5 * season_length:]
        seasonal = slidingSTL_seasonal[5 * season_length:]
        residual = slidingSTL_residual[5 * season_length:]
        
        trend_smooth = utility_stl.calculate_smoothness(slidingSTL_trend)
        """
        Number of lags to use in the Ljung-Box test is min(2m, n/5), where n is the length of the series, and m is the seasonal period of the data. (Forecasting: Principles and Practice 2nd))
        """
        randomness = sm.stats.acorr_ljungbox(slidingSTL_residual,
                                                 lags=[min(2 * season_length, round(len(slidingSTL_residual) / 5))],return_df=True)
        lb_stat = randomness.iloc[0]['lb_stat']
        """
        Kruskal–Wallis test is the second test for stable seasonality [1].
        References
        [1] Bee Dagum, E., Bianconcini, S. (2016). Linear Filters Seasonal Adjustment Methods: Census Method II and Its Variants. In: Seasonal Adjustment Methods and Real Time Trend-Cycle Estimation. Statistics for Social and Behavioral Sciences. Springer, Cham. https://doi.org/10.1007/978-3-319-31822-6_4
        """
        
        sub_seasonalities = [[seasonal[i] for i in range(j, len(seasonal), season_length)] for j in range(season_length)]

        kruskal_test = stats.kruskal(*sub_seasonalities)
        kruskal_pvalue = kruskal_test[1]
        results.append({'algorithm': 'slidingSTL',
                    'dataset': os.path.basename(json_file),
                    'bound': bound,
                    'trend_smooth': trend_smooth,
                    'seasonality_presence': kruskal_pvalue,
                    'randomness': lb_stat})
        
        

/Users/thanapolphungtua-eng/source_code/SSTD_PKDD/dataset/02_Real1_datasets/04_fma_lynx.json _ 10
/Users/thanapolphungtua-eng/source_code/SSTD_PKDD/dataset/02_Real1_datasets/03_co2.json _ 12
/Users/thanapolphungtua-eng/source_code/SSTD_PKDD/dataset/02_Real1_datasets/02_WalkJogRun2.json _ 80
/Users/thanapolphungtua-eng/source_code/SSTD_PKDD/dataset/02_Real1_datasets/01_WalkJogRun1.json _ 80
/Users/thanapolphungtua-eng/source_code/SSTD_PKDD/dataset/02_Real1_datasets/05_SOI_index.json _ 60
/Users/thanapolphungtua-eng/source_code/SSTD_PKDD/dataset/02_Real1_datasets/06_sunspots.json _ 132
